---
cdt: 2024-09-13T08:36:53
title: Extracting Binary Pump Tables
description: Needed to get the solvent composition and timetable information from samples in order to calculate gradients. To do so I needed to create a parsing module. This notebook contains the execution code to extract those tables from the 'raw' dataset. 
conclusion: "A module 'bin_pump_to_db' was created, with associated tests. binary pump tables can now be found in the 'bin_pump' schema."
project: database_architecture
---


In [ ]:
# environment

%reload_ext autoreload
%autoreload 2
from pca_analysis.code.bin_pumps_to_db import bin_pump_to_db

import duckdb as db
from pca_analysis.experiments.constants import db_path
from pathlib import Path

con = db.connect(db_path)



# Bin Pump Tables to DB

Have created a module to extract the binary pump tables to the database.


In [ ]:
ddir = Path("/Users/jonathan/uni/0_jono_data/raw_uv")
wine_deg_samples = Path("/Users/jonathan/uni/0_jono_data/wine-deg-study/raw_uv/ambient").glob("2023-04-21_A*.D")
paths = list(ddir.glob("*.D")) + list(wine_deg_samples)
_ = [bin_pump_to_db(str(path), con) for path in paths]

con.sql(
    """--sql
SELECT
    *
FROM
    bin_pump.solvcomps
LIMIT 3
"""
).pl().pipe(display)

con.sql(
    """--sql
SELECT
    *
FROM
    bin_pump.timetables
LIMIT 5
"""
).pl().pipe(display)

con.sql(
    """--sql
SELECT
    *
FROM
    bin_pump.id
LIMIT 3
"""
).pl().pipe(display)


In [ ]:
con.sql(
"""--sql

"""
)
con.sql(
"""--sql
"""
)

con.sql(
"""--sql
SELECT
    table_name as name,
    COMMENT
FROM
    duckdb_tables
WHERE
    table_name = 'id'
UNION
SELECT
    column_name as column,
    COMMENT
FROM
    duckdb_columns
WHERE
    schema_name = 'bin_pump'
AND
    table_name = 'id'
AND
    column_name = 'id'
OR
    column_name = 'tbl_num'
"""
).pl()


In [ ]:
# con.sql(
# """--sql
# DROP SCHEMA bin_pump CASCADE;
# """
# )


# Conclusion

The sample binary pump data has been successfully extracted. To use the tables, join id's to the primary 'id' table then use the 'tbl_num' key to get the data from the 'solvcomps' or 'timetables' tables.